# Compute the Resource Requirements for Gene Expression in Cell-Free Systems
In this practicum, we will do a proof-of-concept study that computes the resource requirements for gene expression in cell-free systems. We will use a simplified model of the metabolic reactions occurring during transcription and translation.

__Case__: let's consider a simple model of inducible gene expression in a cell-free system operating in a well-mixed batch volume where a single target gene is expressed following the addition of an inducer $I$. We'll assume that we are operating a [reconstituted in-vitro transcription/translation system such as PURExpress](https://www.neb.com/en-us/applications/protein-expression/cell-free-protein-expression/purexpress?srsltid=AfmBOorYg5kvpcMTOG1CrUK3lDBXRcFJVGwEuXMuYr9zeyaKBf7pNQo_) which does not contain any endogenous genes, or metabolic enzymes to generate energy or precursors. The system is assumed to be a batch system, i.e., we are not adding any additional resources after the initial setup.

## Tasks
Before we get started, execute the `Run All Cells` command to check if you have any code or setup issues. Code issues, post a question on EdDiscussion.
* __Task 1: Build the Stoichiometric and Default Bounds Matrices__: In this task, we build the system matrices, i.e., the stoichiometric matrix and the default bounds array for the cell-free system. These will be _sequence-specific_ matrices, i.e., built (on the fly) for a specific sequence we want to express. 
*  __Task 2: Update the Bounds Array__: In this task, we update the default bounds array to reflect the bounds for the cell-free system. In particular, we know that there are pathologies when using flux balance analysis to model gene expression, e.g., translation occurs without transcription, and transcription occurs maximally without inducer $I$, etc. We can fix these problems by being clever about the bounds for the reactions in the cell-free system. 
* __Task 3: Estimate the Steady-State Cell-Free Transcription and Translation Rates__: In this task, we will estimate the rate of transcription and translation for our protein of interest in a cell-free system as a function of system parameters, such as inducer concentration. This will generate a set of steady-state fluxes for the system that we can explore.

Let's get started! (Don't forget to answer the discussion questions!)

___

## Setup, Data, and Prerequisites
Before we get going, let's set up the computational environment by including the `Include.jl` file, loading any needed resources, such as sample datasets, and setting up any required constants. 
* The `Include.jl` file also loads external packages, various functions that we will use in the exercise, and custom types to model the components of our problem. It checks for a `Manifest.toml` file; if it finds one, packages are loaded. Other packages are downloaded and then loaded.

In [1]:
include("Include.jl");

### Constants
Let's set up some constants that we will use in the exercise. The comments in the code provide more details on each constant, its purpose, its value, etc.

In [ ]:
R = 8.314; # universal gas constant (units: J/(mol*K))
T = 37 + 273.15; # temperature (units: K);
β = 1/(R*T); # inverse temperature (units: 1/(J*mol)) - thermodynamic beta in units of 1/(J*mol)
μ = 0.0; # in this case no growth, we are cell free!
parameters = generate_parameter_dictionary(joinpath(_PATH_TO_CONFIGURATION, "Parameters.json")); # load the biophysical parameters (approximately true)
INDUCER = 1.0; # initial concentration of the inducer (units: mM)
KIX = 10.0; # Saturation constant for the Hill function in the u-function (units: mM)

## Task 1: Build the Stoichiometric and Default Bounds Matrices
In this task, we will build the system matrices, i.e., the stoichiometric matrix and the default bounds array for the cell-free system. These will be _sequence-specific_ matrices, i.e., built (on the fly) for a specific sequence we want to express. 

Let's start by loading the gene and protein sequences used in the exercise. The gene sequence is contained in the [`deGFP.gene` file](data/deGFP.gene), and the protein sequence is included in the [`deGFP.prot` file](data/deGFP.protein). The gene sequence is a DNA sequence, and the protein sequence is an amino acid sequence. We load these files using [the `load_gene_sequence_from_file(...)` method](src/Files.jl), while the protein sequence is loaded using the [the `load_protein_sequence_from_file(...)` method](src/Files.jl). 
* These methods take a path to the file as an argument, and return a `genesequence::String` and `proteinsequence::String` variable, respectively.

Both sequences [are Strings](https://docs.julialang.org/en/v1/manual/strings/), so we'll need to process them in a bit.

In [3]:
genesequence, proteinsequence = let

    # setup the paths to the gene and protein sequence files
    path_to_gene_file = joinpath(_PATH_TO_DATA, "deGFP.gene") # test gene
    path_to_protein_file = joinpath(_PATH_TO_DATA, "deGFP.prot") # test protein

    # load the sequences from the files - 
    gene_sequence = load_gene_sequence_from_file(path_to_gene_file); # load the gene sequence
    protein_sequence = load_protein_sequence_from_file(path_to_protein_file); # load the protein sequence

    # return -
    gene_sequence, protein_sequence
end;

__Generate metabolic transcription and translation reactions:__ In the code block below, we will generate the transcription and translation reactions for the gene and protein sequences. In addition, we create exchange reactions for every species in the system. We store the reactions in the `reactions::Vector{String}` variable. 
* The reactions are stored as strings, and we will parse them later to build the stoichiometric matrix and the default bounds array.
The transcription and translation reactions are generated using the [the `transcription(...)` method](src/Sequence.jl) while the translation reactions are constructed using [the `translation(...)` method](src/Sequence.jl). Finally, the exchange reactions are generated using [the `exchange(...)` method](src/Sequence.jl).
* _What are the transcription and translation reactions?_ Check out [the publication](https://www.biorxiv.org/content/10.1101/139774v1.full.pdf) for a specific description of these reactions. However, in short, these are metabolic reactions that occur during transcription and translation, including the consumption of ATP, GTP, and other metabolites, tRNA charging, and the production of mRNA and protein.

What's cool in this case is that the transcription and translation reactions are generated using the gene and protein sequences, respectively. This means we can generate the reactions for any gene or protein sequence, and the stoichiometric matrix and default bounds array will be built on the fly.

In [4]:
reactions = let

    # initialize -
    reactions = Vector{String}();
    
    # TXTL and "hypothetical" exchange reactions -
    transcription_reactions = transcription(genesequence); # transcription reactions
    translation_reactions = translation(proteinsequence); # translation reactions
    txtl_reactions = [transcription_reactions; translation_reactions]; # combine transcription and translation reactions
    
    # build the exchange reactions -
    exchange_reactions = exchangereactions(txtl_reactions); # exchange reactions
    
    # build the list of reactions -
    reactions = [txtl_reactions; exchange_reactions]; # combine all reactions
end;

__Write VFF reactions to file:__ In the code block below, we will write the reactions to a file in VFF format, and then use some code from lecture and the previous computational exercises to parse the VFF file and build the stoichiometric matrix and default bounds array.

In [5]:
let

    # initialize -
    path_to_reaction_file = joinpath(_PATH_TO_DATA, "deGFP.reactions"); # path to the reaction file

    # write the reactions to the file -
    open(path_to_reaction_file, "w") do io
        for reaction in reactions
            println(io, reaction); # write the reaction to the file
        end
    end
end

__Build the stoichiometric matrix and default bounds array:__ In the code block below, we build the stoichiometric matrix and default bounds array from reaction list using the [the `build_stoichiometric_matrix(...)` method](src/Stoichiometric.jl) and the [the `build_default_bounds_array(...)` method](src/Stoichiometric.jl). These methods take the reactions as input and return the stoichiometric matrix and default bounds array, respectively.
* _What is getting returned from this block?_ The variables that returned from this block are the stoichiometric matrix `S::Array{Float64,2}`, the default bounds array `boundsarray::Array{Float64,2}`, the reactions `reactions::Vector{String}`, the list of species `species::Vector{String}`, and the reaction dictionary `rd::Dict{String, Int64}`.

In [6]:
S, species, reactions, rd, boundsarray = let

    # initialize -
    path_to_reaction_file = joinpath(_PATH_TO_DATA, "deGFP.reactions"); # path to the reaction file
    reactions = read_reaction_file(path_to_reaction_file); # read the reactions from the file
    
    # Compute the stoichiometric matrix -
    (S, species_array, reaction_array, reaction_dictionary) = build_stoichiometric_matrix(reactions); # compute the stoichiometric matrix
   
    # compute the bounds -
    bounds = build_default_bounds_array(reactions); # this is the default bounds array

    # return -
    S, species_array, reactions, reaction_dictionary, bounds
end;

__Problem model__: To store all the problem data, we create an instance of [the `MyPrimalFluxBalanceAnalysisCalculationModel` type](src/Types.jl) and store it in the `model::MyPrimalFluxBalanceAnalysisCalculationModel` variable. We also return a bunch of stuff related to the indexing of the reactions (used to make tables, looking up the reaction index given the name, etc.).
* _Builder (or factory) pattern_: For all custom types that we make, we'll use something like [the builder software pattern](https://en.wikipedia.org/wiki/Builder_pattern) to construct and initialize these objects. The calling syntax will be the same for all types: [a `build(...)` method](src/Factory.jl) will take the kind of thing we want to build in the first argument, and the data needed to make that type as [a `NamedTuple` instance](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple) in the second argument.
* _What's the story with the `let` block_? A [let block](https://docs.julialang.org/en/v1/manual/variables-and-scoping/#Let-Blocks) creates a new hard scope and new variable bindings each time it runs. Thus, they act like a private scratch space, where data comes in (is captured by the block), but only what we want to be exposed comes out.

In [7]:
model, reactionnamesmap, inversereactionsnamemap, rnamesarray = let

    # build the FBA model -
    model = build(MyPrimalFluxBalanceAnalysisCalculationModel, (
        S = S, # stoichiometric matrix
        fluxbounds = boundsarray, # these are the *default* bounds; we'll need to update with new info if we have it
        species = species, # list of species. The rows of S are in this order
        reactions = reactions, # list of reactions. The cols of S are in this order
        objective = length(reactions) |> R -> zeros(R), # this is empty, we'll need to set this
    ));

    # build a reaction names = reaction index map
    reactionnamesmap = Dict{String,Int64}()
    inversereactionsnamemap = Dict{Int64,String}();
    for i ∈ eachindex(reactions)
        reactionstring = reactions[i]; # get the reaction string
        components = split(reactionstring, ","); # split the reaction string into components around ,
        rname = components[1]; # get the reaction name
        reactionnamesmap[rname] = i;
        inversereactionsnamemap[i] = rname; # add the reaction name to the inverse map
    end

    # build the reaction names array -
    rnamesarray = Vector{String}(undef, length(reactions)); # initialize the reaction names array
    for i ∈ eachindex(reactions)
        reactionstring = reactions[i]; # get the reaction string
        components = split(reactionstring, ","); # split the reaction string into components around ,
        rname = components[1] |> String; # get the reaction name
        rnamesarray[i] = rname; # add the reaction name to the array
    end

    # return -
    model, reactionnamesmap, inversereactionsnamemap, rnamesarray
end;

## Task 2: Update the Bounds Array
In this task, we update the default bounds array to reflect the bounds for the cell-free system. In particular, we know that there are pathologies when using flux balance analysis to model gene expression, e.g., translation occurs without transcription, and transcription occurs maximally without inducer $I$, etc. We fix these problems by setting the bounds for the reactions in the cell-free system.

There is a trick with the bounds (that incorporates many things we explored in class) that we can use to fix the gene expression problem:
* [Vilkhovoy M, Horvath N, Shih CH, Wayman JA, Calhoun K, Swartz J, Varner JD. Sequence-Specific Modeling of E. coli Cell-Free Protein Synthesis. ACS Synth Biol. 2018 Aug 17;7(8):1844-1857. doi: 10.1021/acssynbio.7b00465. Epub 2018 Jul 16. PMID: 29944340.](https://www.biorxiv.org/content/10.1101/139774v2)
* __Fix__: We (equality) bound the transcription rate $\hat{v}_{i}$ as $\hat{r}_{X, i}u_{i} = \hat{v}_{i} = \hat{r}_{X, i}u_{i}$, while a translation rate is bounded from above by the modified kinetic limit: $0\leq\hat{v}_{j}\leq\hat{r}_{L,j}w_{j}$. This is interesting because the kinetic limits are (semi)mechanistic descriptions of the transcription and translation rate. At the same time, the control variables contain Boltzmann-type descriptions of the logical control of these processes. Thus, we have a mechanistic and logical description of the transcription and translation rates.

Let's implement this for our gene expression model. First, compute the kinetic limit of transcription for your gene of interest [using the `compute_transcription_rate(...)` method](src/Utility.jl). This method takes the `parameters::Dict{Symbol, Any}` dictionary as input, and returns the _kinetic limit_ of transcription, which we store in the  `rX::Float64` variable. 

In [8]:
rX = compute_transcription_rate(parameters); # transcription kinetic limit

In [9]:
println("Maximim transcription rate (mu/mol-hr): ", rX); # print the transcription rate

Maximim transcription rate (mu/mol-hr): 0.20993100642270118


Next, compute the transcriptional and translational control terms that govern how inducer $I$ drives gene expression. These control terms attenuate the kinetic limit expressions. For simplicity, let's assume the translation control term is unity $w = 1$ and focus on a Boltzmann-type description of the transcriptional control term $u$. 

For our proof-of-concept promoter, we assume three states:
* __State 0: Bare gene__: This state is just the gene without anything bound to it. This state will __not__ lead to transcription. This will be our ground state. The pseudo energy for this state $\epsilon_{0} \equiv 0$ J/mol.
* __State 1: RNAP only__: Only RNAP is bound to the promoter without an inducer bound. This state __will__ lead to (background) transcription at a low level. The pseudo energy for this state $\epsilon_{1} \approx 3474$ J/mol.
* __State 2: RNAP + I__: In this state both RNAP and inducer $I$ are bound to the promoter. This state __will__ lead to transcription. The pseudo energy for this state $\epsilon_{2} \approx -14,707$ J/mol.

In the code block below, we compute $u_{1}$ (the control variable for state 1, background expression), $u_{2}$ (the control variable for state 2, induced expression), and translation control parameter $w$. For a reference on this topic, see the Supplemental materials of [Moon et al.](https://pubmed.ncbi.nlm.nih.gov/23041931/) and/or our lecture `L6c` materials.

In [10]:
u₁, u₂, w = let 
    
    # initialize -
    w = 1.0;
    u₁ = 0.0; # this is udagger (background)
    u₂ = 0.0; # this is u (induced)
    ϵₒ = 0.0; # pseudo-energy for the background (gene only) state 0
    ϵ₁ = 3474.0; # pseudo-energy for state 1 (gene + RNAP) - units: J/mol
    ϵ₂ = -14707.0; # pseudo-energy for state 2 (gene + RNAP + P_PhoB) - units: J/mol

    # we'll model the f₂ function as the fraction of PhoB binding. Others are at defaults
    fₒ = 1.0; # default value
    f₁ = 1.0; # default value
    f₂ = INDUCER/(INDUCER + KIX); # hmmm. This is sort of interesting ...

    # compute the weights of each state
    Wₒ = fₒ*exp(-β*ϵₒ); # TODO: Update the weight term for state 0
    W₁ = f₁*exp(-β*ϵ₁); # TODO: Update the weight term for state 1
    W₂ = f₂*exp(-β*ϵ₂); # TODO: Update the weight term for state 2
    Z = Wₒ + W₁ + W₂; # partition function

    # Boltzmann promoter logic here -
    u₁ = W₁/Z; # TODO: Update u₁ (this is u dagger in the notes)
    u₂ = W₂/Z; # TODO: Update u₂ (this is u)
    
    # return -
    u₁,u₂,w
end;

In [11]:
println("u₁ (background expression factor): $(u₁) and u₂ (induced expression) $(u₂)"); # print the u₁ and u₂ value

u₁ (background expression factor): 0.06146297872016685 and u₂ (induced expression) 0.7020993915068997


Finally, let's compute the kinetic limit of translation. This one is tricky because it requires an estimate of the _concentration_ of the mRNA for the gene of interest (flux balance analysis doesn't have concentrations, only flux). Let's approximate the mRNA level by the steady-state level (written for transcript $j$):
$$
\begin{align*}
 m^{\star}_{j} & = \frac{r_{X,j}u_{j}\left(\dots\right) + \lambda_{j}}{\theta_{m,j}+\mu}\quad\text{for }j=1,2,\dots,N
\end{align*}
$$
where $\lambda_{j} \equiv r_{X,j}u^{\dagger}_{j}$. See lecture `L5b` for a description of the kinetic limit of translation expression (or the reference we gave above). We save the kinetic limit of translation in the `rL::Float64` variable, and the steady-state mRNA level in the `m̄::Float64` variable.

In [12]:
rL, m̄ = let

    # get constants from parameters -
    θ = parameters[:kdX]; # first order degradation constant mRNA (units: 1/hr)
    KL = parameters[:KL]; # saturation constant translation (units: μmol/gDW-hr)
    VMAXL = parameters[:VL]; # VMAX translation (correct PhoA length) (units: μmol/gDW-hr)
    τ = parameters[:L_tau_factor]; # relative time constant translation (units: dimensionless)
    
    # compute -
    m = (rX*u₁ + rX*u₂)/(θ + μ); # TODO: approx mRNA level with steady-state
    rL = VMAXL*(m/(τ*KL+(1+τ)*m)); # compute the kinetic limit of translation

    # return -
    rL, m
end;

In [13]:
println("Maximim translation rate (mu/mol-hr): ", rL); # print the translation rate

Maximim translation rate (mu/mol-hr): 0.0902214998897148


### Update the bounds
Now that we have an estimate of the transcription $\hat{r}_{X}u$ and the translation $\hat{r}_{L}w$ rates, we can update the flux balance analysis problem bounds. `Unhide` the code block below to see how we set the bounds. 
* _What is going on here?_ We are setting the bounds for the transcription and translation reactions to equal the kinetic limits modified by the control variables. We are also setting the bounds for specific exchange reactions, e.g., the exchange of charged tRNA to equal zero (we are forcing the system to run the tRNA reactions). Finally, we are setting the upper bound of the mRNA degradation rate to be equal to $\sim\theta\cdot{\bar{m}}$, where $\theta$ is the degradation rate and $\bar{m}$ is the mRNA level, i.e., we assume degradation follows first-order kinetics.

We save the updated bounds in the `fluxbounds::Array{Float64,2}` variable.

In [14]:
fluxbounds = let

    # make a copy of the default flux bounds -
    flux_bounds = copy(model.fluxbounds);

    # what is the the bound on transcription?
    XB = rX*(u₁ + u₂); # this is the lower bound on transcription

    # update the bounds for gene exprssion -
    flux_bounds[reactionnamesmap["transcription_test"], 1] = XB; # transcrption lower bound state 1
    flux_bounds[reactionnamesmap["transcription_test"], 2] = XB; # transcrption upper bound state 1
    flux_bounds[reactionnamesmap["translation_initiation_test"], 1] = rL*w; # translation lower bound
    flux_bounds[reactionnamesmap["translation_initiation_test"], 2] = rL*w; # translation upper bound
    flux_bounds[reactionnamesmap["translation_test"], 1] = 0.0; # translation lower bound
    flux_bounds[reactionnamesmap["translation_test"], 2] = rL*w; # translation upper bound

    # update bounds on tRNA - no exchange of charged tRNA
    tRNA_exchange_reactions = findall(x-> (contains(x, "exchange") && contains(x,"tRNA") && contains(x, "_M_")), rnamesarray); # find the index of the reaction
    for i ∈ eachindex(tRNA_exchange_reactions)
        flux_bounds[tRNA_exchange_reactions[i], 1] = 0.0; # no charged tRNA from box
        flux_bounds[tRNA_exchange_reactions[i], 2] = 0.0; # no charged tRNA into box
    end

    # bound mRNA degradation -
    θ = parameters[:kdX]; # first order degradation constant mRNA (units: 1/hr)
    flux_bounds[reactionnamesmap["mRNA_degradation_test"], 1] = 0.0 # lower bound on mRNA degradation
    flux_bounds[reactionnamesmap["mRNA_degradation_test"], 2] = θ*m̄; # upper bound on mRNA degradation

    # no mRNA exchange -
    flux_bounds[reactionnamesmap["exchange_mRNA_test"], 1] = 0.0; # lower bound on mRNA exchange
    flux_bounds[reactionnamesmap["exchange_mRNA_test"], 2] = 0.0; # upper bound on mRNA exchange

    # ATP exchange -
    # exchange_M_atp_c,[],M_atp_c,true
    # TODO: DQ2: update the bounds on ATP exchange to simulate resource limitation
    flux_bounds[reactionnamesmap["exchange_M_atp_c"], 1] = -1000.0; # lower bound on ATP exchange
    flux_bounds[reactionnamesmap["exchange_M_atp_c"], 2] = 1000.0; # upper bound on ATP exchange

    # TODO: DQ3: update the bounds on ala exchange to simulate resource limitation
    # exchange_M_ala_L_c,[],M_ala_L_c,true
    flux_bounds[reactionnamesmap["exchange_M_ala_L_c"], 1] = -1000.0; # lower bound on ala exchange
    flux_bounds[reactionnamesmap["exchange_M_ala_L_c"], 2] = 1000.0; # upper bound on ala exchange

    # return new bounds
    flux_bounds;
end;

In [15]:
model.fluxbounds = fluxbounds; # update the flux bounds in the model

## Task 3: Estimate the Protein Production Rate
In this task, we will estimate the transcription and translation rate for our protein of interest in a cell-free system as a function of system parameters, such as inducer concentration. Let's begin by updating the objective function. In [a previous study](https://www.biorxiv.org/content/10.1101/139774v1.full.pdf), we found that a good objective for this type of problem is to maximize the protein translation rate. We do that here:

In [16]:
objective_coefficients = let

    # initialize -
    nreactions = length(model.reactions); # how many reactions do we have?
    objective_coefficients = zeros(nreactions); # initialize the objective coefficients

    # which reactions do we want to maximize?
    reactions_to_maximize = ["translation_test"]; # TODO: Add reactions that we want to max, lets max translation_test
    for i ∈ eachindex(reactions_to_maximize)
        reaction = reactions_to_maximize[i];
        j = reactionnamesmap[reaction];
        objective_coefficients[j] = 1;
    end

    # return -
    objective_coefficients;
end;

In [17]:
model.objective = objective_coefficients; # update the objectivec coefficients

### Compute the optimal flux distribution 
We have updated the bounds array to reflect the bounds for the cell-free system and set the objective function to be the rate of protein translation. Now, we can compute the optimal flux distribution.

Let's compute the optimal metabolic distribution $\left\{\hat{v}_{i} \mid i = 1,2,\dots,\mathcal{R}\right\}$ by solving the [linear programming problem using the `GLPK.jl` package](https://github.com/jump-dev/GLPK.jl). We solve the optimization problem by passing the `model::MyPrimalFluxBalanceAnalysisCalculationModel` to [the `solve(...)` method](src/Compute.jl). This method returns a `solution::Dict{String, Any}` dictionary, which holds information about the solution.
* __Why the [try-catch environment](https://docs.julialang.org/en/v1/base/base/#try)__? The [solve(...) method](src/Compute.jl) has an [@assert statement](https://docs.julialang.org/en/v1/base/base/#Base.@assert) to check if the calculation has converged. Thus, the solve method can [throw](https://docs.julialang.org/en/v1/base/base/#Core.throw) an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) if the optimization problem fails to converge. To gracefully handle this case, we use a [try-catch construct](https://docs.julialang.org/en/v1/base/base/#try). See the [is_solved_and_feasible method from the JuMP package](https://jump.dev/JuMP.jl/stable/api/JuMP/#JuMP.is_solved_and_feasible) for more information.

In [18]:
solution = let
    
    solution = nothing; # initialize nothing for the solution
    try
        solution = solve(model); # call the solve method with our problem model -
    catch error
        println("error: $(error)"); # Oooooops! Looks like we have a *major malfunction*, problem didn't solve
    end

    # return solution
    solution
end;

### Flux table

If the optimization problem converged, we should have the optimal flux values throughout the system. Let's use [the `pretty_tables(...)` method exported by the `PrettyTables.jl` package](https://github.com/ronisbr/PrettyTables.jl) to display the estimated optimal metabolic fluxes. `Unhide` the code block below to see how we constructed the flux table.

In [19]:
let

    # setup -
    S = model.S;
    flux_bounds_array = model.fluxbounds;
    number_of_reactions = size(S,2); # columns
    flux = solution["argmax"];

	# nonzero fluxes -
	nonzero_fluxes = findall(v-> abs(v) ≥ 1e-6, flux); # find the non-zero fluxes
	number_of_nonzero_fluxes = length(nonzero_fluxes); # how many non-zero fluxes do we have?
    
    # populate the state table -
	flux_table = Array{Any,2}(undef,number_of_nonzero_fluxes,5)
	for reaction_index ∈ eachindex(nonzero_fluxes)
		i = nonzero_fluxes[reaction_index]; # get the index of the reaction
		flux_table[reaction_index,1] = inversereactionsnamemap[i]
		flux_table[reaction_index,2] = flux[i]
		flux_table[reaction_index,3] = flux_bounds_array[i,1]
		flux_table[reaction_index,4] = flux_bounds_array[i,2]
        flux_table[reaction_index,5] = inversereactionsnamemap[i] |> key-> rd[key]
	end

    # header row -
	flux_table_header_row = (["Reaction","v̂ᵢ", "v̂ᵢ LB", "v̂ᵢ UB", "Reaction"],["","μmol/L-time", "μmol/L-time", "μmmol/L-time", "N/A"]);
		
	# write the table -
	pretty_table(flux_table; header=flux_table_header_row, tf=tf_simple, alignment = :l)
end

=============================== ============= ============= ============== =====================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================
  Reaction                       v̂ᵢ            v̂ᵢ LB         v̂ᵢ UB          Reaction                                                                                                                                                                                                                                                                                                                                                

In [20]:
do_I_see_the_flux_table_flag = true; # TODO: update this flag value {true | false}. true if you see the table, false otherwise

__Want to see a particular flux?__ Look at the names in [the deGFP.reactions file](data/deGFP.reactions) and set this to the `flux_name_I_want_to_see::String` variable. We'll look up that name and then display the flux value.

In [21]:
let

    # initialize -
    flux_name_I_want_to_see = "exchange_M_atp_c"; # TODO: update this to the flux name you want to see
    flux_name_I_want_to_see_index = reactionnamesmap[flux_name_I_want_to_see]; # get the index of the flux
    v = solution["argmax"][flux_name_I_want_to_see_index]; # get the flux value

    # print the flux value -
    println("Flux value for $(flux_name_I_want_to_see): $(v) μmol/L-hr"); # print the flux value
end

Flux value for exchange_M_atp_c: 46.42799042145407 μmol/L-hr


## Discussion
* __DQ1__: One of the hallmarks of inducible gene expression is the ability to control the transcription rate by the concentration of the inducer. 
    - Increase the concentration of the inducer to $I = 1.0$ mM (from our default value of $I = 0.1$ mM). What happens to the rate of transcription (and translation)? Is the response linear? What do you think is going on here?

In [22]:
## -- DQ1 answer goes here -- ##

In [23]:
did_I_answer_DQ1 = false; # TODO: update this flag value {true | false}. true if you answered the question, false otherwise

* __DQ2__: One of the interesting things about this approach is its ability to compute the implications of resource limitations. For example, if we set the ATP exchange reaction to be equal to zero, we could compute the impact of ATP limitation on the rate of transcription and translation. 
    - Update the code block in the bounds block and (re)run the notebook. What do you observe? Could you explain the results (Note: a valid result is that there is no solution)?

In [24]:
## -- DQ2 answer goes here -- ##

In [25]:
did_I_answer_DQ2 = false; # TODO: update this flag value {true | false}. true if you answered the question, false otherwise

* __DQ3__: Reset the ATP exchange to the default values ($\pm{1000}$) and instead set the alanine (or any of the other amino acids) exchange rates to zero (which simulates amino acid limitation). 
    - Update the code in the bounds block for the alanine exchange and (re)run the notebook. What do you observe? Could you explain the results (Note: a valid result is that there is no solution)?

In [26]:
## -- DQ3 answer goes here -- ##

In [27]:
did_I_answer_DQ3 = false; # TODO: update this flag value {true | false}. true if you answered the question, false otherwise

## Fun (totally optional) directions we could go with this idea in the future
This proof-of-concept study used flux balance analysis to compute the resource requirements for gene expression in cell-free systems. There are many directions we could go with this idea in the future. Here are a few ideas:
* __Use a more complex model of the metabolic network__: The model we used in this exercise is a simplified model of the metabolic network. We could use a more complex model that includes the reactions required to produce the amino acids and energy that the system consumes. This would allow us to compute the resource requirements for gene expression in a more realistic system. For laughs, I've included a more complex model of the [metabolic network in the `data` directory](data/Metabolism.net). This model is based on the _E. coli_ metabolic network includes the reactions required to produce the amino acids and energy from glucose.

* __Incorporate more information in the bounds array__: The bounds array we used in this exercise is a simplified version of the bounds array used in flux balance analysis. We could have gotten more granular about computing the upper bounds for various reactions, e.g., the exchange reactions, the charging of tRNA, etc. Thus, we could also have included information about the kinetics of the reactions, e.g., the Michaelis-Menten kinetics for the reactions, and used enzyme kinetic data, and metabolite measurements to constrain the system further. Better bounds estimates give us better estimates of the fluxes.

* __Simulate the expression of synthetic circuits in cell-free systems__: The model can be used to simulate the expression of multiple genes. For example, we could use this model to simulate the expression of a synthetic circuit that includes multiple genes and compute the resource requirements for the entire circuit. The model could be a quick and dirty way to calculate the resource requirements for a synthetic circuit, and could be used to guide the design of the circuit.

If we only had more time, we could do all these things. But alas, we are out of time!

## Tests
`Unhide` the code block below (if you are curious) about how we implemented the tests and what we are testing. In these tests, we check values in your notebook and give feedback on which items are correct, missing etc.

In [28]:
let
    @testset verbose = true "CHEME 5450 practicum test suite" begin
        
        @testset "Setup" begin
            @test isnothing(model) == false
            @test isnothing(rd) == false
            @test isnothing(reactionnamesmap) == false
        end

        @testset "Problem, Bounds and Optmization" begin
            @test rX != 0.0; # kinetic limit of transcription should be non-zero
            @test rL != 0.0; # kinetic limit of translation should be non-zero
            @test w == 1.0; # default value 
            @test u₁ != 0.0; # background state should be non-zero
            @test u₂ != 0.0; # induced state should be non-zero            
            @test isempty(solution) == false # solution should not be empty
            @test do_I_see_the_flux_table_flag == true # do I see the flux table?
        end
        
        @testset "Discussion questions" begin
             @test did_I_answer_DQ1 == true
             @test did_I_answer_DQ2 == true
             @test did_I_answer_DQ3 == true
        end
    end
end;

Discussion questions: Test Failed at /Users/jeffreyvarner/Desktop/julia_work/CHEME-5820-SP25/CHEME-5450-Practicum-S2025/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X66sZmlsZQ==.jl:21
  Expression: did_I_answer_DQ1 == true
   Evaluated: false == true

Stacktrace:
 [1] macro expansion
   @ ~/.julia/juliaup/julia-1.11.5+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Test/src/Test.jl:679 [inlined]
 [2] macro expansion
   @ ~/Desktop/julia_work/CHEME-5820-SP25/CHEME-5450-Practicum-S2025/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X66sZmlsZQ==.jl:21 [inlined]
 [3] macro expansion
   @ ~/.julia/juliaup/julia-1.11.5+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Test/src/Test.jl:1704 [inlined]
 [4] macro expansion
   @ ~/Desktop/julia_work/CHEME-5820-SP25/CHEME-5450-Practicum-S2025/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X66sZmlsZQ==.jl:21 [inlined]
 [5] macro expansion
   @ ~/.julia/juliaup/julia-1.11.5+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Test/src/Tes

TestSetException: Some tests did not pass: 10 passed, 3 failed, 0 errored, 0 broken.